In [1]:
import os
import csv
from time import *

import sys
sys.path.insert(0, "../scripts/")

from graphreader import read_graph

from greedy import greedy_mwds, get_successors

In [2]:
dir_instances = sorted(os.listdir("../instances"))

In [4]:
from pulp import *

def lp_model(vertices_w, edges):
    vertices = list(vertices_w.keys())

    model = LpProblem("Minimum Weighted Directed Domination Set", LpMinimize)

    # definisemo varijable za cvorove u grafu
    x = LpVariable.dicts("x", vertices, lowBound=0, upBound=1, cat=LpInteger)

    # dodajemo funkciju cilja
    model += lpSum([vertices_w[i] * x[i] for i in vertices])

    # dodajemo ogranicenja
    # Svaki cvor mora biti u podskupu ili imati vezu sa nekim cvorom iz tog podskupa
    for v in vertices:
        constraint = x[v]
        for e in edges:
            if e[1] == v:
                constraint += x[e[0]]
        model += constraint >= 1


    # pokrecemo model, sa ogranicenjem na 10 minuta
    time_limit_s = 600
    model.solve(PULP_CBC_CMD(msg=0, maxSeconds=time_limit_s))

    # kreiramo dominirajuci skup
    domination_set = []
    for i in vertices:
      if x[i].value() == 1:
        domination_set.append(i)

    objective_value = value(model.objective)

    # vracamo dominirajuci skup i tezinu
    return domination_set, objective_value


In [5]:
# prolazak kroz razlicite dimenzije problema
for subdir in dir_instances:
    dim_path = os.path.join("../instances/", subdir)
    print("Dimension - ", subdir)

    if subdir.endswith("0"):
        instance_dim = "0"
        num_vertices = 5
    elif subdir.endswith("1"):
        instance_dim = "1"
        num_vertices = 300
    else:
        instance_dim = "2"
        num_vertices = 500

    with open("../results/results" + instance_dim + ".csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)

        # header
        writer.writerow(["Instances", "Greedy algorithm", "Variable neighborhood search (VNS)", "PuLP linar programming"])

        # prolazak kroz sve instance odredjene dimenzije
        for instance in sorted(os.listdir(os.path.abspath(dim_path))):
            instance_path = os.path.join(dim_path, instance)
            vertices, edges = read_graph(instance_path, num_vertices)
            num_edges = len(edges)

            start = time()
            set, weight = greedy_mwds(vertices, edges)
            end = time()

            elapsed = end - start
            greedy_results = "|Ds| = " + str(len(set)) + "\n" + "weight = " + str(weight) +"\n" + "T(s) = " + f"{elapsed:.3f}"

            start = time()
            set, weight = lp_model(vertices, edges)
            end = time()

            elapsed = end - start
            ilp_results = "|Ds| = " + str(len(set)) + "\n" + "weight = " + str(weight) +"\n" + "T(s) = " + f"{elapsed:.3f}"

            instance_info = "Instance " + instance_dim + "lvl\n" + str("|V| = ") + str(num_vertices) + "\n|E| = " + str(num_edges)
            writer.writerow([instance_info, greedy_results, 0, ilp_results])


Dimension -  level0
Dimension -  level1


/home/vladimir/anaconda3/envs/operaciona_istrazivanja/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/home/vladimir/anaconda3/envs/operaciona_istrazivanja/lib/python3.10/site-packages/pulp/apis/coin_api.py:95: UserWarning: Parameter maxSeconds is being depreciated for timeLimit
  warnings.warn("Parameter maxSeconds is being depreciated for timeLimit")


Dimension -  level2
